In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#from matplotlib import rcParams
#import scipy.stats as st
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import RandomizedSearchCV # Model evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Dropout, BatchNormalization
#from keras.layers.advanced_activations import PReLU
#from keras.layers.noise import GaussianDropout
#from keras.optimizers import Adam
#from keras.wrappers.scikit_learn import KerasRegressor
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import Imputer

In [ ]:
### read data and process categorical data
modified_data_path = "D:/Columbia U/IEOR 4501 python/Project/data_modified.csv"

data = pd.read_csv(modified_data_path)

print("logPrice Min")
print(data['log_price'].min())
print("logPrice Mean")
print(data['log_price'].mean())
print("logPrice Median")
print(data['log_price'].median())
print("logPrice Max")
print(data['log_price'].max())
print("logPrice Std")
print(data['log_price'].std())

# data = data.merge(pd.get_dummies(data.property_type, drop_first=True, prefix='PT'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.room_type, drop_first=True, prefix='RT'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.bed_type, drop_first=True, prefix='BT'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.cancellation_policy, drop_first=True, prefix='CP'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.city, prefix='City'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.host_has_profile_pic, drop_first=True, prefix='HHPP'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.host_identity_verified, drop_first=True, prefix='HHPV'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.instant_bookable, drop_first=True, prefix='IB'), left_index=True, right_index=True)
# data = data.merge(pd.get_dummies(data.neighbourhood, drop_first=True, prefix='NE'), left_index=True, right_index=True)

# del data['property_type']
# del data['room_type'],
# del data['bed_type']
# del data['cancellation_policy']
# del data['city']
# del data['host_has_profile_pic']
# del data['host_identity_verified']
# del data['instant_bookable']
# del data['neighbourhood']
# del data['Saleprice']

for c in data.columns:
    if data[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(data[c].values))
        data[c] = lbl.transform(list(data[c].values))


In [ ]:
### prepare train data set and test data set for analysis

data.drop(['id'], axis=1, inplace=True)
X = data.drop("log_price",axis=1).values
y = data["log_price"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)

In [ ]:
#select features using threshold
thresh = 5 * 10**(-2)
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
selection = SelectFromModel(model, threshold=thresh, prefit=True)
select_X_train = selection.transform(X_train)
# test 
select_X_test = selection.transform(X_test)


In [ ]:
### Test the scores for XGBoost
xgb1 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

xgb1.fit(X_train, y_train)

y_hat = xgb1.predict(X_test)

xgb1.score(X_test,y_test)

n_folds = 10
kfold = KFold(n_splits=n_folds, random_state=42)
cv_results = cross_val_score(model, X_train, y_train, cv= kfold,scoring='r2', n_jobs=-1)

In [ ]:
### XGBoost ###

xgb_params = {
    'eta': 0.03,
    'max_depth': 10,
    'subsample': 0.75,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'lambda': 0.8,   
    'alpha': 0.4,
    'silent': 1
}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

num_boost_rounds = 200
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred1 = model.predict(dtest)

print(mean_absolute_error(xgb_pred1, y_test))
print(mean_squared_error(xgb_pred1, y_test))

print(pd.DataFrame(xgb_pred1).head())

xgb_params = {
    'eta': 0.03,
    'max_depth': 10,
    'subsample': 0.75,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(select_X_train, y_train)
dtest = xgb.DMatrix(select_X_test)

num_boost_rounds = 200

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred2 = model.predict(dtest)

print(mean_absolute_error(xgb_pred2, y_test))

print(mean_squared_error(xgb_pred2, y_test))

In [ ]:
####################
### RandomForest ###
####################


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(select_X_train, y_train)

rf_random.best_params_

y_hat = rf_random.predict(select_X_test)

median_absolute_error(y_hat, y_test)

np.sqrt(MSE(y_hat, y_test))